In [1]:
import scipy.io
import torch
import importlib
import matplotlib.pyplot as plt

In [2]:
import src.HyperParameters as hp 
import src.data as dt 
import src.model as Model 
import src.train as trainer

importlib.reload(hp)
importlib.reload(dt)
importlib.reload(Model)
importlib.reload(trainer)

<module 'src.train' from '/Users/ramtarun/Desktop/Cambridge/Friction-Factor-Estimation-PINN/src/train.py'>

In [3]:
data = scipy.io.loadmat('/Users/ramtarun/Desktop/Cambridge/Indirect-Noise-in-Nozzles/Data/Data_PINN_subsonic_geom_linvelsup_f0-0.1.mat')

In [4]:
# inputs, targets, meanflow =  dt.DataPreprocessing(data, ff=0.01)
# train_loader, val_loader = dt.DataTransformer(inputs, targets, meanflow, TrainingSet=True)


In [5]:
device = torch.device('cpu')

In [6]:
device

device(type='cpu')

In [7]:
PINN_model = Model.PINN()

In [8]:
params = list(PINN_model.parameters())
optimizer = torch.optim.Adam([{'params' : params[1::]},{'params' : params[0], 'lr': hp.ff_learning_rate}], lr = hp.learning_rate, amsgrad = True)   

# optimizer = torch.optim.LBFGS(params, hp.ff_learning_rate, 
#                               max_iter = hp.epochs, 
#                               max_eval = None, 
#                               tolerance_grad = 1e-11, 
#                               tolerance_change = 1e-11, 
#                               history_size = 100, 
#                               line_search_fn = 'strong_wolfe')


In [9]:
fval = [0.1  , 0.06 , 0.04, 0.01]

for f in fval:
    inputs, targets, meanflow =  dt.DataPreprocessing(data, ff=f, device=device)
    N = inputs.shape[0]
    train_loader, val_loader = dt.DataTransformer(inputs, targets, meanflow, TrainingSet=True)
    trainer.train(train_loader, val_loader, hp.epochs, optimizer, PINN_model, N)


Epoch 1500/7500 - Train Loss: 0.000074 Val Loss: 0.000897 f_train: 0.0659 f_test: 0.0659


: 

: 

In [ ]:
ff_test = [0.08, 0.02, 0.005, 0.000]
test_loss = {}
ff_est = {}
for f_test in ff_test:

    inputs, targets, meanflow =  dt.DataPreprocessing(data, ff=f_test)
    test_loader = dt.DataTransformer(inputs, targets, meanflow, TrainingSet=False)
    with torch.no_grad():
        val_loss = 0.0
        for k, (A, B, C) in enumerate(test_loader):
            u_pred = PINN_model.dnn(A)
            val_loss += torch.linalg.norm((B-u_pred),2)/torch.linalg.norm(B,2)
            #val_loss += PINN_model.Loss(A, B, C).item()

        val_loss /= len(val_loader) # Average Validation Loss
        test_loss[k] = val_loss

    plt.figure()
    plt.plot(inputs[:,1], targets, 'k')
    plt.plot(inputs[:,1], PINN_model.dnn(inputs).detach(), 'r-.')


In [ ]:
PINN_model.f